In [2]:
import pandas as pd
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import SentenceTransformerEmbeddings
import re


In [3]:
df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/data/10data.csv", sep=",", encoding="utf-8")

def normalize_text(text):
    if pd.isna(text):
        return ""
    text = text.replace("\n", " ")
    text = text.replace("\r", " ")
    text = text.replace('"', '')
    text = re.sub(r"\.{2,}", "", text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    text = text.lower()
    return text

df["rawText_clean"] = df["rawText"].apply(normalize_text)


In [3]:
emb_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")


/tmp/ipykernel_3738686/993254723.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_model = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")
/home/yapayzeka/ahsen_bulbul/ahsen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
semantic_chunker = SemanticChunker(
    embeddings=emb_model,
    buffer_size=30,                    # chunk büyüklüğü ayarı
    breakpoint_threshold_type="gradient",
    breakpoint_threshold_amount=0.5,   
    number_of_chunks=10,                # her metin için max chunk sayısı
    min_chunk_size=50
)

# Örnek olarak ilk kararın textini chunk et
text = df.loc[0, "rawText_clean"]
chunks = semantic_chunker.split_text(text)

# Chunk’leri yazdır
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk[:200]}...")  # sadece ilk 200 karakteri göster


Chunk 1: 6. hukuk dairesi 2023/596 e. , 2024/257 k. \n i̇çtihat metni mahkemesi̇ : bölge adliye mahkemesi 13. hukuk dairesi taraflar arasındaki rücuen tazminat davasından dolayı yapılan yargılama sonunda, i̇lk...


In [10]:
all_chunks = []
for index, row in df.iterrows():
    text = row["rawText_clean"]
    chunks = semantic_chunker.split_text(text)
    for chunk in chunks:
        all_chunks.append({
            "id": row["_id"],
            "chunk_text": chunk
        })

# Pandas DataFrame olarak saklayabilirsin
chunks_df = pd.DataFrame(all_chunks)
chunks_df.to_csv("chunks2.csv", index=False, encoding="utf-8")
